<a href="https://colab.research.google.com/github/AlexWortega/Collab_experement/blob/main/2chgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [1]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3MB 6.6MB/s 
     |████████████████████████████████| 1.1MB 18.5MB/s 
     |████████████████████████████████| 2.9MB 25.4MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0fd00ce1a7dd0e4bc4006940cfd8110a365a1d052ee41952cda086ef1fe77a38
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
pip install dvach.api

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for addict: filename=addict-2.2.0-cp37-none-any.whl size=2783 sha256=b41584888ab6b954752a960f6a44542966e19b4448cbc657e744e23d26750874
  Stored in directory: /root/.cache/pip/wheels/1c/35/37/7a99ee58a72dc257689fb4f99c8018c9513555055e3fb87123
  Created wheel for simplejson: filename=simplejson-3.16.0-cp37-cp37m-linux_x86_64.whl size=114846 sha256=4aeeccb996de50a6c3c808f0b92c27dac6808a18b5a0a5b6ffdd55857d4e7000
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Successfully built addict simplejson
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: idna 2.10
 

In [3]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 595 (delta 56), reused 36 (delta 18), pack-reused 505
Receiving objects: 100% (595/595), 349.36 KiB | 4.31 MiB/s, done.
Resolving deltas: 100% (356/356), done.


In [4]:
!mkdir models/

## Download files

In [5]:
import api2ch
brds= ['b','d','sn','cc','soc','po','un','o','po']
ch = []
for b in brds:

  api = api2ch.DvachApi(b)

  board = api.get_board()

  print(f'Total {len(board)} threads in {api.board.id}')
  for thread in board:
      ch+=[(thread.post.comment)]
    


Total 210 threads in b
Total 136 threads in d
Total 193 threads in sn
Total 356 threads in cc
Total 129 threads in soc
Total 172 threads in po
Total 78 threads in un
Total 140 threads in o
Total 172 threads in po


In [6]:
len(ch)

1586

In [7]:
val = ch[:len(ch)//2]
train = ch[len(ch)//2:] 
with open("val.txt", "w") as txt_file:
    for line in val:
        txt_file.write("".join(line) + "\n")
with open("train.txt", "w") as txt_file:
    for line in train:
        txt_file.write("".join(line) + "\n")

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [12]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=val.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

2021-03-06 17:35:43.537138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/06/2021 17:35:45 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
03/06/2021 17:35:52 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, block_size=2048, 

## Evaluate model
### Generate with generate_transformers.py script.

In [14]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/essays \
    --k=5 \
    --p=0.95 \
    --length=50 \
    --repetition_penalty=5

In [19]:
!git clone https://github.com/AlexWortega/Collab_experement

Cloning into 'Collab_experement'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 30 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [20]:
cd /content/Collab_experement

/content/Collab_experement


In [29]:
!git commit --message="add 2ch model"

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
	./

nothing added to commit but untracked files present


In [30]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address


In [28]:
!git config --global user.email "alexdragannikolich@gmail.com"
!git config --global user.name "AlexWortega"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Generate mannualy

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
text = "<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

In [ ]:
tok.decode(out[0])